In [1]:
from pathlib import Path
from tqdm import tqdm
from typing import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


ROOT_DIR = Path(os.getcwd()).parent

In [2]:
df: pd.DataFrame = pd.read_parquet(
    os.path.join(ROOT_DIR, "data/datasets/train_01_05.parquet")
)
df = df.reset_index(drop=True)

df["pump_hash"] = df["pumped_ticker"] + "_" + df["pump_time"]
df["is_pumped"].value_counts()

is_pumped
False    19534
True       136
Name: count, dtype: int64

<h4>Fill nans</h4>

In [3]:
df.isna().sum().sort_values(ascending=False).to_frame().head(40)

,0
quote_slippage_imbalance_ratio_1h_std_14d,2884
quote_slippage_imbalance_ratio_1h_std_7d,1791
quote_slippage_imbalance_ratio_1h,1506
quote_slippage_imbalance_ratio_1h_std_3d,894
quote_slippage_imbalance_ratio_1h_mean_14d,702
quote_slippage_imbalance_ratio_3h,470
quote_slippage_imbalance_ratio_1h_std_1d,418
quote_slippage_imbalance_ratio_1h_mean_7d,253
quote_slippage_imbalance_ratio_7h,163
long_trades_ratio_1h,105


In [14]:
df.iloc[:5, -10:]

,quote_slippage_imbalance_ratio_12h,quote_slippage_quote_abs_ratio_12h,num_prev_pumps,exchange,pumped_ticker,pump_time,ticker,is_pumped,days_listed,pump_hash
0,0.006160,0.000284,0,binance,BRDBTC,2018-12-22 17:00:00,VIBBTC,False,418,BRDBTC_2018-12-22 17:00:00
1,0.683209,0.002124,0,binance,BRDBTC,2018-12-22 17:00:00,NULSUSDT,False,152,BRDBTC_2018-12-22 17:00:00
2,0.084946,0.000107,0,binance,BRDBTC,2018-12-22 17:00:00,ETHUSDT,False,492,BRDBTC_2018-12-22 17:00:00
3,-0.671166,0.001180,0,binance,BRDBTC,2018-12-22 17:00:00,GVTBTC,False,401,BRDBTC_2018-12-22 17:00:00
4,-0.159345,0.001340,0,binance,BRDBTC,2018-12-22 17:00:00,REQBTC,False,421,BRDBTC_2018-12-22 17:00:00


In [4]:
reg_cols = df.columns[:-7]

In [5]:
# fill in nans with zeros for other cols

df[reg_cols] = df[reg_cols].fillna(0)

In [6]:
cols_fill_median = [
    "daily_exchange_volume_share_3d_mean",
    "daily_exchange_volume_share_7d_mean",
    "daily_exchange_volume_share_14d_mean",
    "daily_exchange_volume_share_30d_mean",
    "daily_exchange_volume_share_30d_std",
    "daily_exchange_volume_share_3d_std",
    "daily_exchange_volume_share_14d_std",
    "daily_exchange_volume_share_7d_std",
]

In [8]:
# fill in nans with median values
df_features: pd.DataFrame = pd.DataFrame()

for (pumped_ticker, pump_time), df_pump in df.groupby(["pumped_ticker", "pump_time"]):
    pump_has_nans: bool = df_pump[df_pump["ticker"] == pumped_ticker][cols_fill_median].isna().any().any()

    if pump_has_nans:
        continue

    for col in cols_fill_median:
        df_pump[col] = df_pump[col].fillna(df_pump[col].median())

    df_features = pd.concat([df_features, df_pump])

df_features["is_pumped"].value_counts()

KeyError: "None of [Index(['daily_exchange_volume_share_3d_mean',\n       'daily_exchange_volume_share_7d_mean',\n       'daily_exchange_volume_share_14d_mean',\n       'daily_exchange_volume_share_30d_mean',\n       'daily_exchange_volume_share_30d_std',\n       'daily_exchange_volume_share_3d_std',\n       'daily_exchange_volume_share_14d_std',\n       'daily_exchange_volume_share_7d_std'],\n      dtype='object')] are in the [columns]"

In [9]:
df_pumps: pd.DataFrame = pd.DataFrame()

for i, (pump_hash, df_pump) in enumerate(df_features.groupby("pump_hash")):
    if i < 3:
        df_pumps = pd.concat([df_pumps, df_pump])

KeyError: 'pump_hash'

In [ ]:
fig, axs = plt.subplots(nrows=10, ncols=5, figsize=(30, 15))
axs = [el for ax in axs for el in ax]

for ax, col in tqdm(zip(axs, reg_cols[50:100]), total=len(reg_cols[50:100])):
    try:
        sns.histplot(
            data=df_pumps, x=col, hue="pump_hash", alpha=.3, legend=False, ax=ax, kde=True
        )
    except:
        pass
    
plt.tight_layout()
plt.show()

<h4>Apply crosssectional normalization</h4>

In [ ]:
# reg_cols = list(set(reg_cols) - set(cols_fill_median))

df_features = df.copy()

In [ ]:
df_features_norm: pd.DataFrame = pd.DataFrame()
cols_to_standardize: List[str] = list(set(reg_cols) - set(["num_prev_pumps"]))

for pump_hash, df_pump in df_features.groupby("pump_hash"):
    for col in cols_to_standardize:
        df_pump[col] = (df_pump[col] - df_pump[col].mean()) / df_pump[col].std()

    df_features_norm = pd.concat([df_features_norm, df_pump])

In [ ]:
df_features

In [ ]:
df_train: pd.DataFrame = df_features_norm[
    df_features_norm["pump_time"] < "2020-05-01"
].copy()

df_test: pd.DataFrame = df_features_norm[
    df_features_norm["pump_time"] >= "2020-05-01"
].copy()

df_train["is_pumped"].value_counts(), df_test["is_pumped"].value_counts()

<h4>Train the model</h4>

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, KFold
from sklearn.metrics import f1_score, precision_recall_curve, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

from functools import partial
from catboost import Pool, CatBoostClassifier

import optuna
import gc

<h4>Vanilla CatboostClassifier</h4>

In [ ]:
train: Pool = Pool(data=df_train[reg_cols], label=df_train["is_pumped"], cat_features=["num_prev_pumps"])
test: Pool = Pool(data=df_test[reg_cols], label=df_test["is_pumped"], cat_features=["num_prev_pumps"])

model_baseline = CatBoostClassifier(
    objective="Logloss", iterations=200, verbose=100
)
model_baseline.fit(train)

y_proba: np.array = model_baseline.predict_proba(test)

In [ ]:
df_test["proba"] = y_proba[:, 1]

thresholds = range(1, 301)
baseline_top_k_vals = []

for K in thresholds: 

    top_k: List[bool] = []

    for pump_hash, df_pump in df_test.groupby("pump_hash"):
        df_pump = df_pump.sort_values(by="proba", ascending=False)
        top_k_contains_pump: bool = df_pump.iloc[:K]["is_pumped"].any()
        top_k.append(top_k_contains_pump)

    baseline_top_k_vals.append(
        sum(top_k) / len(top_k)
    )

auc(x=np.linspace(0, 1, 300), y=baseline_top_k_vals)

In [ ]:
plt.plot(thresholds, baseline_top_k_vals)
plt.show()

<h4>Optimize for Top-K AUC</h4>

In [ ]:
def calc_top_k_auc(df_test: pd.DataFrame, y_proba: np.array) -> Tuple[float, np.array]:
    """Returns the area under the TOP-K curve"""
    df_test: pd.DataFrame = df_test.copy()
    df_test["proba"] = y_proba

    thresholds = range(1, 301)
    top_k_vals = []

    for K in thresholds: 

        top_k: List[bool] = []

        for pump_hash, df_pump in df_test.groupby("pump_hash"):
            df_pump = df_pump.sort_values(by="proba", ascending=False)
            top_k_contains_pump: bool = df_pump.iloc[:K]["is_pumped"].any()
            top_k.append(top_k_contains_pump)

        top_k_vals.append(
            sum(top_k) / len(top_k)
        )

    return auc(x=np.linspace(0, 1, 300), y=top_k_vals), top_k_vals

In [ ]:
def catboost_objective(
    trial: optuna.Trial, df: pd.DataFrame, reg_cols: List[str], target: str, fold: TimeSeriesSplit
) -> float:
    params = {
        "objective": "Logloss",
        "eval_metric": "Logloss",
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        "class_weights": [1, trial.suggest_float("scale_pos_weight", 1, 300)],
    }

    auc_scores = []
    
    i = 0
    
    for train_idx, val_idx in fold.split(df[reg_cols], df[target]):
        # split data to train and validation sets
        df_train, df_val = df.iloc[train_idx], df.iloc[val_idx]
        # train on train subset and use validation set to evaluate the model
        train = Pool(data=df_train[reg_cols], label=df_train[target], cat_features=["num_prev_pumps"])
        val = Pool(data=df_val[reg_cols], label=df_val[target], cat_features=["num_prev_pumps"])
        
        # Fit the model with early stopping
        model = CatBoostClassifier(
            **params, 
            iterations=500,
            early_stopping_rounds=20,
            use_best_model=True,
            verbose=False
        )

        model.fit(train, eval_set=val)
        
        y_proba = model.predict_proba(val)[:, 1]
        top_k_auc, _ = calc_top_k_auc(df_test=df_val, y_proba=y_proba)
        
        # get the best auc_score validation set
        auc_scores.append(top_k_auc)
        
        trial.report(top_k_auc, i)

        if trial.should_prune():
            raise optuna.TrialPruned()
        
        del model, train, val
        _ = gc.collect()
        
        i += 1

    return np.mean(auc_scores)

In [ ]:
fold = TimeSeriesSplit(n_splits=5)

study_catboost = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5)
)

study_catboost.optimize(
    partial(catboost_objective, df=df_train, reg_cols=reg_cols, target="is_pumped", fold=fold), 
    n_trials=80
)

In [ ]:
params = {
    "objective": "Logloss",
    # "task_type": "GPU"
}

cb_params = {
    "objective": "Logloss",
    # "task_type": "GPU"
}

cb_params.update(study_catboost.best_params)
cb_params["class_weights"] = [1, study_catboost.best_params["scale_pos_weight"]]

del cb_params["scale_pos_weight"]

cb_params

In [ ]:
model = CatBoostClassifier(
    # **cb_params,
    iterations=300,
    # early_stopping_rounds=20,
    # use_best_model=True,
    verbose=100
)
model.fit(
    train, 
    # eval_set=val
)

y_proba: np.array = model.predict_proba(test)[:, 1]

In [ ]:
top_k_auc, top_k_vals = calc_top_k_auc(df_test, y_proba=y_proba)

plt.plot(np.linspace(0, 1, 300), baseline_top_k_vals, label="Baseline model")
plt.plot(np.linspace(0, 1, 300), top_k_vals, label=f"Tuned Catboost: {round(top_k_auc, 4)}")

plt.title("TOP-K AUC")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix

y_pred = (y_proba >= 0.05).astype(int)

print(classification_report(y_true=df_test["is_pumped"], y_pred=y_pred))

In [ ]:
cm = confusion_matrix(y_pred=y_pred, y_true=df_test["is_pumped"])
ConfusionMatrixDisplay(confusion_matrix=cm).plot()
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, auc, PrecisionRecallDisplay

precision, recall, thresholds = precision_recall_curve(y_true=df_test["is_pumped"], probas_pred=y_proba)

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)

PrecisionRecallDisplay(
    precision=precision, recall=recall
).plot(ax=ax)

f_scores = np.linspace(0.1, 0.8, num=10)
lines, labels = [], []

for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    (l,) = ax.plot(x[y >= 0], y[y >= 0], color="blue", alpha=0.2)
    ax.annotate("f1={0:0.1f}".format(f_score), xy=(0.9, y[45] + 0.02))

plt.title(f"AUC score: {round(auc(recall, precision), 5)}")
plt.show()

In [ ]:
# TOP-K metric
df_test["proba"] = y_proba

top_k_vals = []

for K in [1, 5, 10, 20, 40, 70]: 

    top_k: List[bool] = []

    for pump_hash, df_pump in df_test.groupby("pump_hash"):
        df_pump = df_pump.sort_values(by="proba", ascending=False)
        top_k_contains_pump: bool = df_pump.iloc[:K]["is_pumped"].any()
        top_k.append(top_k_contains_pump)

    top_k_vals.append(
        sum(top_k) / len(top_k)
    )
        
top_k_vals

In [ ]:
df_fi = pd.DataFrame({
    "features": reg_cols,
    "feature_importance": model.feature_importances_
})

df_fi = df_fi.sort_values(by="feature_importance", ascending=False)


fig, axs = plt.subplots(1, 2, figsize=(20, 10))
ax1, ax2 = axs

sns.barplot(
    data=df_fi.iloc[:50], x="feature_importance", y="features", ax=ax1
)
sns.barplot(
    data=df_fi.iloc[-50:], x="feature_importance", y="features", ax=ax2
)

plt.tight_layout()
plt.show()